In [1]:
import numpy as np
import pandas as pd
import pickle
import spacy
import re

In [2]:
es_index = pickle.load(open("../wordvecs/es_index.pkl", 'rb'))

In [3]:
len(es_index)

985667

In [4]:
vocab_set = set(es_index.keys())

In [5]:
df = pd.read_pickle("../data/elmondo_es_qa_no_numbers.pkl")

In [6]:
df.head()

,question,answer,story
0,La información proviene de los documentos del...,@entity534,La @entity852 (@entity123) trabaja en la const...
1,Han detenido a 7 personas intregrantes de la ...,@entity919,Esta semana @entity681 ha recibido una noticia...
2,@placeholder tiene una gran tasa de criminali...,@entity756,Esta semana @entity756 ha recibido una noticia...
3,Los civiles armados se niegan y se registran ...,@entity733,La situación en el @entity222 de @entity174 es...
4,"Su líder moral, el dr. @placeholder, dice en ...",@entity711,La situación en el @entity205 de @entity457 es...


In [7]:
nlp = spacy.load('es')

In [8]:
def encode_text(text_str):
    doc = nlp(text_str, parse=False, tag=False, entity=False)
    out = []
    for t in doc:
        w = t.text.lower()
        if w in vocab_set:
            out.append(es_index[w])
        elif w[:7] == '@number':
            out.append(-1)
        elif w[:7] == '@entity':
            num = int(re.search('\d+', w).group(0))
            out.append(-100 - num)
        elif w[:12] == '@placeholder':
            out.append(-2)
        elif not re.search('\w+', w):
            continue
        else:
            out.append(-3)
    return out

In [9]:
df['question'][:10].apply(encode_text)

0      [4, 366, 2663, 0, 15, 2144, 14, 575, 17774, -2]
1    [145, 4885, 10, -3, 160, 659096, 0, 4, 235, 70...
2    [-2, 80, 21, 82, 3414, 0, 37263, -1, 6759, 17,...
3    [15, 2215, 6582, 16, 22505, 7, 16, 16119, 5771...
4    [22, 1108, 3331, 2, 6, 1213, 3, -2, 2, 561, 5,...
5    [22, 1108, 3331, 2, 6, 1213, 3, -379, 2, 561, ...
6    [70, 22, 6216, 29, 58, 618, 2, 3900, 7, 4585, ...
7    [6, 6481, -2, 2, 5097, 5, -302, 2, 3143, 0, 74...
8    [6, 6481, -822, 2, 5097, 5, -2, 2, 3143, 0, 74...
9    [6, 6481, -257, 2, 5097, 5, -266, 2, 3143, 0, ...
Name: question, dtype: object